In [4]:
import os, json

from others.utils import read_airland_file, generate_separation_between_runways
from models.MIP import solve_single_runway_mip, solve_multiple_runways_mip
from models.CP import solve_single_runway_cp, solve_multiple_runways_cp
from models.CP_MIP import solve_hybrid_lbbd

# MIP

In [ ]:
data_dir = "data"
output_json = "metrics.json"

num_files = 13
all_metrics = {"MIP Single": []}

for i in range(1, num_files + 1):
    file_name = f"airland{i}.txt"
    file_path = os.path.join(data_dir, file_name)

    print("\n" + "="*60)
    print(f"Processing file: {file_name}")
    print("="*60 + "\n")

    data = read_airland_file(file_path)

    solver, variables, metrics = solve_single_runway_mip(
        data['p'], data['planes'], data['separation_times'], hint=True, performance=True)

    metrics_with_file = {"file": file_name, **metrics}
    all_metrics["MIP Single"].append(metrics_with_file)

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table:")
print("{:<12} {:<15} {:<15} {:<15} {:<15} {:<15}".format(
    "File", "Exec Time(s)", "Variables", "Constraints", "Total Penalty", "B&B Nodes"
))
for m in all_metrics["MIP Single"]:
    print("{:<12} {:<15.4f} {:<15} {:<15} {:<15} {:<15}".format(
        m["file"], m["execution_time"], m["num_variables"],
        m["num_constraints"], m["total_penalty"], m["num_branch_and_bound_nodes"]
    ))



Processing file: airland1.txt

		Creating Single Runway MIP Model

-> Decision variables: 120
-> Constraints: 205

			Solving MIP

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest
-------------------------------------------------
    0 |       165.00 |   129.00 | 155.00 | 559.00
    1 |       258.00 |   195.00 | 258.00 | 744.00
    2 |        98.00 |    89.00 |  98.00 | 510.00
    3 |       106.00 |    96.00 | 106.00 | 521.00
    4 |       118.00 |   110.00 | 123.00 | 555.00
    5 |       126.00 |   120.00 | 135.00 | 576.00
    6 |       134.00 |   124.00 | 138.00 | 577.00
    7 |       142.00 |   126.00 | 140.00 | 573.00
    8 |       150.00 |   135.00 | 150.00 | 591.00
    9 |       180.00 |   160.00 | 180.00 | 657.00

-> Planes that did not land on the target time:
Plane | Landing Time | Target | Early Dev | Late Dev | Penalty
--------------------------------------------------------------
    0 |       165.00 | 155.00 |      0.00 |    10.00 |  100.0

In [ ]:
data_dir = "data"
output_json = "metrics.json"

all_metrics = {
    "MIP Single": all_metrics.get("MIP Single", []),
    "MIP Multiple": []
}

num_files = 13

for i in range(1, num_files + 1):
    file_name = f"airland{i}.txt"
    file_path = os.path.join(data_dir, file_name)

    print("\n" + "="*60)
    print(f"Processing file: {file_name}")
    print("="*60 + "\n")

    data = read_airland_file(file_path)

    separation_times_between_runways = generate_separation_between_runways(
        data['p'], 2, separation_same_runway=True, default_between_runways=2
    )
    r = 2
    solver, variables, metrics = solve_multiple_runways_mip(
        data['p'], r, data['planes'], data['separation_times'], separation_times_between_runways,
        hint=True, performance=True
    )

    metrics_with_file = {"file": file_name, **metrics}
    all_metrics["MIP Multiple"].append(metrics_with_file)

with open(output_json, "w") as f:
    json.dump(all_metrics, f, indent=4)

print(f"\n-> Metrics saved to {output_json}")

# Metrics Table
print("\nMetrics Table (MIP Multiple):")
print("{:<12} {:<15} {:<15} {:<15} {:<15} {:<15}".format(
    "File", "Exec Time(s)", "Variables", "Constraints", "Total Penalty", "B&B Nodes"
))
for m in all_metrics["MIP Multiple"]:
    print("{:<12} {:<15.4f} {:<15} {:<15} {:<15} {:<15}".format(
        m["file"], m["execution_time"], m["num_variables"],
        m["num_constraints"], m["total_penalty"], m["num_branch_and_bound_nodes"]
    ))



Processing file: airland1.txt

		Creating Multiple Runways MIP Solver

-> Decision variables: 230
-> Constraints: 350

			Solving MIP

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest | Runway
----------------------------------------------------------
    0 |       155.00 |   129.00 | 155.00 | 559.00 |      1
    1 |       258.00 |   195.00 | 258.00 | 744.00 |      0
    2 |        98.00 |    89.00 |  98.00 | 510.00 |      1
    3 |       106.00 |    96.00 | 106.00 | 521.00 |      0
    4 |       123.00 |   110.00 | 123.00 | 555.00 |      0
    5 |       132.00 |   120.00 | 135.00 | 576.00 |      1
    6 |       138.00 |   124.00 | 138.00 | 577.00 |      0
    7 |       140.00 |   126.00 | 140.00 | 573.00 |      1
    8 |       150.00 |   135.00 | 150.00 | 591.00 |      0
    9 |       180.00 |   160.00 | 180.00 | 657.00 |      1

-> Planes that did not land on the target time:
Plane | Landing Time | Target | Early Dev | Late Dev | Penalty | Runway
---

# CP

In [5]:
data_dir = "data"

num_files = 7
for i in range(1, num_files + 1):
    file_name = f"airland{i}.txt"
    file_path = os.path.join(data_dir, file_name)

    print("\n" + "="*60)
    print(f"Processing file: {file_name}")
    print("="*60 + "\n")

    data = read_airland_file(file_path)

    solver, variables, models = solve_single_runway_cp(
        data['p'], data['planes'], data['separation_times'], hint=True)


Processing file: airland1.txt

		     Creating CP model

-> Number of decision variables created: 75
-> Number of constraints: 130

			Solving CP

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest
-------------------------------------------------
    0 |       165.00 |   129.00 | 155.00 | 559.00
    1 |       258.00 |   195.00 | 258.00 | 744.00
    2 |        98.00 |    89.00 |  98.00 | 510.00
    3 |       106.00 |    96.00 | 106.00 | 521.00
    4 |       118.00 |   110.00 | 123.00 | 555.00
    5 |       126.00 |   120.00 | 135.00 | 576.00
    6 |       134.00 |   124.00 | 138.00 | 577.00
    7 |       142.00 |   126.00 | 140.00 | 573.00
    8 |       150.00 |   135.00 | 150.00 | 591.00
    9 |       180.00 |   160.00 | 180.00 | 657.00

-> Planes that did not land on the target time:
Plane | Landing Time | Target | Early Dev | Late Dev | Penalty
--------------------------------------------------------------
    0 |       165.00 | 155.00 |      0.00 |  

In [6]:
data_dir = "data"

num_files = 7
for i in range(1, num_files + 1):
    file_name = f"airland{i}.txt"
    file_path = os.path.join(data_dir, file_name)

    print("\n" + "="*60)
    print(f"Processing file: {file_name}")
    print("="*60 + "\n")

    data = read_airland_file(file_path)

    separation_times_between_runways = generate_separation_between_runways(data['p'], 2, separation_same_runway=True, default_between_runways=2)
    r = 2
    solver, variables, models = solve_multiple_runways_cp(
        data['p'], r, data['planes'], data['separation_times'], separation_times_between_runways, hint=True)


Processing file: airland1.txt

		     Creating CP model

-> Number of decision variables created: 130
-> Number of constraints: 310

			Solving CP

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest | Runway
----------------------------------------------------------
    0 |       155.00 |   129.00 | 155.00 | 559.00 |      0
    1 |       258.00 |   195.00 | 258.00 | 744.00 |      0
    2 |        98.00 |    89.00 |  98.00 | 510.00 |      0
    3 |       106.00 |    96.00 | 106.00 | 521.00 |      0
    4 |       123.00 |   110.00 | 123.00 | 555.00 |      0
    5 |       132.00 |   120.00 | 135.00 | 576.00 |      0
    6 |       138.00 |   124.00 | 138.00 | 577.00 |      1
    7 |       140.00 |   126.00 | 140.00 | 573.00 |      0
    8 |       150.00 |   135.00 | 150.00 | 591.00 |      1
    9 |       180.00 |   160.00 | 180.00 | 657.00 |      1

-> Planes that did not land on the target time:
Plane | Landing Time | Target | Early Dev | Late Dev | Penalty

In [7]:
data_dir = "data"

num_files = 7
for i in range(1, num_files + 1):
    file_name = f"airland{i}.txt"
    file_path = os.path.join(data_dir, file_name)

    print("\n" + "="*60)
    print(f"Processing file: {file_name}")
    print("="*60 + "\n")

    data = read_airland_file(file_path)

    num_runways = 2
    separation_times_between_runways = generate_separation_between_runways(data['p'], 2, separation_same_runway=True, default_between_runways=2)

    solve_hybrid_lbbd(data['p'], num_runways, data['planes'], data['separation_times'], separation_times_between_runways, max_iterations=50)


Processing file: airland1.txt


		Running Hybrid LBBD Solver (Strengthened Master)

--- Iteration 1 ---
  >> Master Theta: 90 | Subproblem Real Cost: 90

*** CONVERGENCE ACHIEVED in 1 iterations! ***

-> Landing times of all planes:
Plane | Landing Time | Earliest | Target | Latest | Runway
----------------------------------------------------------
    0 |       155.00 |   129.00 | 155.00 | 559.00 |      1
    1 |       258.00 |   195.00 | 258.00 | 744.00 |      2
    2 |        98.00 |    89.00 |  98.00 | 510.00 |      1
    3 |       106.00 |    96.00 | 106.00 | 521.00 |      1
    4 |       123.00 |   110.00 | 123.00 | 555.00 |      1
    5 |       134.00 |   120.00 | 135.00 | 576.00 |      2
    6 |       138.00 |   124.00 | 138.00 | 577.00 |      1
    7 |       142.00 |   126.00 | 140.00 | 573.00 |      2
    8 |       150.00 |   135.00 | 150.00 | 591.00 |      2
    9 |       180.00 |   160.00 | 180.00 | 657.00 |      2

-> Planes that did not land on the target time:
Plane | L